### Passo 2 - Limpeza e Tratamento dos Dados

In [4]:
# Importação das Bibliotecas
import os
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import BytesIO
from util.PokeAPIEnrich import enrich_with_pokeapi
from dotenv import load_dotenv

# Definição das Variáveis
load_dotenv("../.env")
connection_string = os.getenv("CONNECTION_STRING")
container_names = ["raw","processed"]
blob_names = ["pokemon.csv","pokemon_finalizado.csv"]
save_path = "../data/"

# Função para remover colunas desnecessárias
def removerColunas(df):
    # Definição das colunas a serem removidas
    columnsToRemove = [
        'abilities','against_bug','against_dark','against_dragon','against_electric',
        'against_fairy','against_fight','against_fire','against_flying','against_ghost',
        'against_grass','against_ground','against_ice','against_normal','against_poison',
        'against_psychic','against_rock','against_steel','against_water','base_egg_steps',
        'base_happiness','classfication','height_m','japanese_name','percentage_male','weight_kg'
    ]

    # Remoção das colunas desnecessárias
    df = df.drop(columns=columnsToRemove)

    # Salvamento dessa atapa da limpeza localmente
    df.to_csv(f"{save_path}pokemon_limpo.csv", index=False)
    print(f"Remoção de colunas desnecessárias concluída e arquivo salvo localmente em {save_path}pokemon_limpo.csv")
    return df

# Função para adicionar as formas do pokémon Minior
def adicionarFormasMinior(df):
    # Seleciona o pokemon "minior" e cria duas cópias da linha
    minior_row = df[df['name'].str.lower() == 'minior']

    minior_meteor = minior_row.copy()
    minior_core = minior_row.copy()

    # Adiciona a informação correta da capture_rate e cria a coluna form
    minior_meteor['form'] = 'meteor'
    minior_meteor['capture_rate'] = 30

    minior_core['form'] = 'core'
    minior_core['capture_rate'] = 255

    # Remove a linha original do dataset
    df = df[df['name'].str.lower() != 'minior'].copy()

    # Preenche com nada nos outros pokemons
    if 'form' not in df.columns:
        df.loc[:, 'form'] = None

    # Adiciona as novas linhas no dataset original
    df = pd.concat([df, minior_meteor, minior_core], ignore_index=True)
    
    # Salvamento dessa atapa da limpeza localmente
    df.to_csv(f"{save_path}pokemon_corrigido.csv", index=False)
    print(f"Remoção de colunas desnecessárias concluída e arquivo salvo localmente em {save_path}pokemon_corrigido.csv")
    return df

# Função para enriquecer o dataset com a PokeAPI
def corrigirVariações(df):
    # Chamada de função em util (ela era grande demais pro Notebook)
    df = enrich_with_pokeapi(df)
    
    # Salvamento dessa atapa da limpeza localmente
    df.to_csv(f"{save_path}pokemon_finalizado.csv", index=False)
    print(f"Remoção de colunas desnecessárias concluída e arquivo salvo localmente em {save_path}pokemon_finalizado.csv")
    return df

# Função para exportar o dataset para o Azure
def exportarParaAzure(df):
    blob_client = blob_service_client.get_blob_client(container=container_names[1], blob=blob_names[1])

    buffer = BytesIO()
    df.to_csv(buffer, index=False)
    buffer.seek(0)

    blob_client.upload_blob(buffer, overwrite=True)
    print("Dataset limpo exportado para o Azure com sucesso.")

# Importação do Dataset crú do Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
blob_client = blob_service_client.get_blob_client(container=container_names[0], blob=blob_names[0])
blob_bytes = blob_client.download_blob().readall()
buffer = BytesIO(blob_bytes)

# Execução das funções
df = pd.read_csv(buffer)
df = removerColunas(df)
df = adicionarFormasMinior(df)
df = corrigirVariações(df)
exportarParaAzure(df)

Remoção de colunas desnecessárias concluída e arquivo salvo localmente em ../data/pokemon_limpo.csv
Remoção de colunas desnecessárias concluída e arquivo salvo localmente em ../data/pokemon_corrigido.csv
Remoção de colunas desnecessárias concluída e arquivo salvo localmente em ../data/pokemon_finalizado.csv
Dataset limpo exportado para o Azure com sucesso.
